In [37]:
import os
import requests
from elasticsearch import Elasticsearch
from tqdm.auto import tqdm
import tiktoken
from mistralai import Mistral


In [3]:
docs_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/01-intro/documents.json?raw=1'
docs_response = requests.get(docs_url)
documents_raw = docs_response.json()

documents = []

for course in documents_raw:
    course_name = course['course']

    for doc in course['documents']:
        doc['course'] = course_name
        documents.append(doc)

In [4]:
es_client = Elasticsearch('http://localhost:9200')

In [5]:
index_settings = {
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 0
    },
    "mappings": {
        "properties": {
            "text": {"type": "text"},
            "section": {"type": "text"},
            "question": {"type": "text"},
            "course": {"type": "keyword"} 
        }
    }
}

index_name = 'course-questions'

es_client.indices.create(index=index_name, body=index_settings)

/tmp/ipykernel_2566/310768491.py:18: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  es_client.indices.create(index=index_name, body=index_settings)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'course-questions'})

In [6]:
for doc in tqdm(documents):
    es_client.index(index=index_name, document=doc)

100%|███████████████| 948/948 [00:05<00:00, 168.05it/s]


In [7]:
def elastic_search(query):
    
    search_query = {
        "size": 3,
        "query": {
            "bool": {
                "must": {
                    "multi_match": {
                        "query": query,
                        "fields": ["question^4", "text^1"],
                        "type": "best_fields"
                    }
                },
                "filter": {
                    "term": {
                        "course": "machine-learning-zoomcamp"
                    }
                }
            }
        }
    }
    
    response = es_client.search(index=index_name, body=search_query)
    
    result_docs = []
    for hit in response['hits']['hits']:
        result_docs.append(hit['_source'])
    
    return result_docs

In [16]:
query = 'How do copy a file to a Docker container?'

search_results = elastic_search(query)

/tmp/ipykernel_2566/139931507.py:23: DeprecationWarning: The 'body' parameter is deprecated and will be removed in a future version. Instead use individual parameters.
  response = es_client.search(index=index_name, body=search_query)


In [18]:
search_results

[{'text': 'Launch the container image in interactive mode and overriding the entrypoint, so that it starts a bash command.\ndocker run -it --entrypoint bash <image>\nIf the container is already running, execute a command in the specific container:\ndocker ps (find the container-id)\ndocker exec -it <container-id> bash\n(Marcos MJD)',
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I debug a docker container?',
  'course': 'machine-learning-zoomcamp'},
 {'text': "You can copy files from your local machine into a Docker container using the docker cp command. Here's how to do it:\nTo copy a file or directory from your local machine into a running Docker container, you can use the `docker cp command`. The basic syntax is as follows:\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\nHrithik Kumar Advani",
  'section': '5. Deploying Machine Learning Models',
  'question': 'How do I copy files from my local machine to docker container?',
 

In [28]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT doent't contain the answer, output NONE

QUESTION: {question}

CONTEXT:
{context}
""".strip()

context = ""

for doc in search_results:
    context = context + f"question: {doc['question']}\nanswer: {doc['text']}\n\n" 

prompt = prompt_template.format(question=query, context=context).strip()

In [30]:
len(prompt)

1537

In [33]:
encoding = tiktoken.encoding_for_model("gpt-4o")

In [35]:
len(encoding.encode(prompt))

334

In [36]:
encoding.decode_single_token_bytes(63842)

b"You're"

In [39]:
api_key = os.environ["MISTRAL_API_KEY"]
client = Mistral(api_key=api_key)

In [40]:
def llm(prompt):
    
    model = "open-mistral-nemo"
    response = client.chat.complete(
    model= model,
    messages = [
        {
            "role": "user",
                "content": prompt,
            },
        ]
    )
    return response.choices[0].message.content

In [42]:
llm(prompt)

"To copy a file to a Docker container, you can use the `docker cp` command. Here's how to do it:\n\n```bash\ndocker cp /path/to/local/file_or_directory container_id:/path/in/container\n```"